In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = True
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    #"red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    #"red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-05 00:07:36,456	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-05 00:07:36,458	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-05 00:07:50,665	INFO trainable.py:130 -- Trainable.setup took 14.212 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-05 00:07:50,666	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-05 00:08:03,989	INFO trainable.py:130 -- Trainable.setup took 13.279 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-05 00:08:03,991	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    #eval_env.reset()
    #results = trainer.train()
    #save_logs(res_name,results,steps,CONTINUAL)
    #print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod,hist_dir)
                    save_hists("red",steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm",steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
298 blue_0 Shoot at red_0 launch distance : 59323.498128835294 True True
311 blue_0 Shoot at red_0 launch distance : 52736.21868843753 True True
358 red_1 Shoot at blue_0
369 red_1 Shoot at blue_0
412 blue_0 Splash :red_0
423 blue_0: Destroyed
1125 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 1125 0.898999897848691 123.20104297013226
blue_1 False False 1125 -0.10700010215130895 12.156929864916068
-------------------------- Scene: 0 --------------------------
880 red_0 Shoot at blue_0
891 red_0 Shoot at blue_0
926 red_1 Shoot at blue_0
937 red_1 Shoot at blue_0
957 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 -0.10099846659283274 -21.301288776924427
blue_1 False False 1200 -0.0009984665928327297 -47.17588964176178
---------

202 blue_1 DOWN
382 red_0 Shoot at blue_0
393 red_0 Shoot at blue_0
407 red_1 Shoot at blue_0
418 red_1 Shoot at blue_0
434 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 434 -1.113001054361241 9.619367609328057
blue_1 False False 434 -0.10100105436124107 -1.505900876740975
-------------------------- Scene: 0 --------------------------
301 blue_0 Shoot at red_1 launch distance : 59602.47440319478 True True
Same tgt shoot
313 blue_0 Shoot at red_0 launch distance : 56988.87103744915 True True
429 blue_0 Splash :red_1
430 blue_0 Splash :red_0
WIN
blue_0 False True 430 28.873444650914095 140.95071665099465
blue_1 False True 430 12.791054902076887 7.450359374418604
-------------------------- Scene: 0 --------------------------
123 blue_0 DOWN
797 red_0 Shoot at blue_1
808 red_0 Shoot at blue_1
820 red_1 Shoot at blue_1
831 red_1 Shoot at blue_1
894 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 894 -0.1009984531958899 -1.2048257697158644
blue_1 False False 894 -0.1435065531958899 -37

WIN
blue_0 False True 540 12.121216255555554 138.77581334737917
blue_1 False True 540 12.221222222222222 118.23707641616123
-------------------------- Scene: 0 --------------------------
361 blue_1 Shoot at red_1 launch distance : 58236.12722574541 True True
Same tgt shoot
385 blue_1 Shoot at red_0 launch distance : 52509.5597689314 True True
414 blue_0 DOWN
426 red_0 Shoot at blue_1
437 red_0 Shoot at blue_1
488 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 488 0.39900010415860604 -4.026368526967968
blue_1 False False 488 0.35889700415860604 63.88023935280443
-------------------------- Scene: 0 --------------------------
487 red_1 Shoot at blue_1
498 red_1 Shoot at blue_1
502 red_0 Shoot at blue_1
513 red_0 Shoot at blue_1
557 blue_1: Destroyed
1017 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 1017 -1.1070006898486364 -43.673709035918534
blue_1 False False 1017 -0.10100068984863637 -17.29971089607237
-------------------------- Scene: 0 --------------------------
298 blue_1 Sh

336 blue_1 Shoot at red_0 launch distance : 58230.535680590474 True True
461 blue_1 Splash :red_0
487 red_1 Shoot at blue_0
498 red_1 Shoot at blue_0
576 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.8990004493472885 -2.9234084800603606
blue_1 False False 1200 0.8990004493472885 32.393166438243256
-------------------------- Scene: 0 --------------------------
185 blue_0 DOWN
375 red_0 Shoot at blue_1
386 red_0 Shoot at blue_1
386 red_1 Shoot at blue_1
397 red_1 Shoot at blue_1
448 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 448 -0.10100035979083972 -1.3046125666158817
blue_1 False False 448 -0.14145395979083975 -13.64308181355713
-------------------------- Scene: 0 --------------------------
720 red_1 Shoot at blue_1
727 red_0 Shoot at blue_1
731 red_1 Shoot at blue_1
738 red_0 Shoot at blue_1
785 blue_0 DOWN
817 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 817 -0.10099842468814772 -14.343605766096157
blue_1 False False 817 -0.14287325802148107 -32.682440

2022-04-05 01:11:47,211	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


...

2022-04-05 10:00:26,371	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000009\checkpoint-9
2022-04-05 10:00:26,371	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 9, '_timesteps_total': 273375, '_time_total': 34729.992698431015, '_episodes_total': 428}


agent_timesteps_total: 546750
custom_metrics: {}
date: 2022-04-05_10-00-26
done: false
episode_len_mean: 615.81
episode_media: {}
episode_reward_max: 310.36215837209863
episode_reward_mean: 153.29989072099937
episode_reward_min: -102.55102456565987
episodes_this_iter: 46
episodes_total: 428
experiment_id: b7bc6bb0bdfa4636b9eb71264eb36a37
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.99474573135376
        entropy_coeff: 0.0
        kl: 0.042333342134952545
        model: {}
        policy_loss: -0.011637118645012379
        total_loss: 178.7132568359375
        vf_explained_var: 0.15207991003990173
        vf_loss: 178.71641540527344
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.853607177734375
        entropy_coeff: 0.0
        kl: 0.0327489972114563
        model

292 blue_1 Shoot at red_1 launch distance : 59483.78996599791 True True
305 blue_1 Shoot at red_1 launch distance : 53360.58765198595 True True
324 blue_0 DOWN
334 red_0 Shoot at blue_1
345 red_0 Shoot at blue_1
401 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 401 0.39900015539004274 -2.326813573178129
blue_1 False False 401 0.35871218872337607 97.66816730201494
-------------------------- Scene: 0 --------------------------
283 blue_1 Shoot at red_0 launch distance : 58759.9947873943 True True
299 blue_1 Shoot at red_0 launch distance : 50603.11399173442 True True
319 blue_0 Shoot at red_0 launch distance : 58269.78305625306 True True
332 blue_0 Shoot at red_0 launch distance : 58682.57510312563 True True
336 red_1 Shoot at blue_1
347 red_1 Shoot at blue_1
395 blue_1 Splash :red_0
407 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.8929914524914128 79.298926435149
blue_1 False False 1200 0.8989990524914128 112.53876774153687
-------------------------- Scene: 0 -----

274 blue_1 Shoot at red_0 launch distance : 58914.28365502992 True True
Same tgt shoot
288 blue_1 Shoot at red_1 launch distance : 58735.52511121766 True True
304 blue_0 DOWN
388 blue_1 Splash :red_0
402 blue_1 Splash :red_1
WIN
blue_0 False True 402 12.885380957262011 11.683659485375385
blue_1 False True 402 29.355530210993354 158.08404059090628
-------------------------- Scene: 0 --------------------------
287 blue_0 Shoot at red_1 launch distance : 59332.87689314704 True True
Same tgt shoot
298 blue_0 Shoot at red_0 launch distance : 55161.44706546517 True True
Same tgt shoot
311 blue_1 Shoot at red_1 launch distance : 57795.394681434096 True True
Same tgt shoot
Same tgt shoot
323 blue_1 Shoot at red_0 launch distance : 53170.131676625555 True True
401 blue_0 Splash :red_1
401 blue_0 Splash :red_0
WIN
blue_0 False True 523 12.294455066921607 136.44073707340527
blue_1 False True 523 12.294455066921607 97.61370416692137
-------------------------- Scene: 0 --------------------------
28

339 red_1 Shoot at blue_1
Same tgt shoot
Same tgt shoot
348 blue_0 Shoot at red_1 launch distance : 53211.146765863756 True True
395 blue_1 Splash :red_0
453 blue_0 Splash :red_1
WIN
blue_0 False True 548 12.18878102189781 118.93204373359745
blue_1 False True 548 12.088781021897809 155.70097505565272
-------------------------- Scene: 0 --------------------------
326 blue_0 Shoot at red_0 launch distance : 59776.26407481486 True True
348 blue_0 Shoot at red_0 launch distance : 50523.25596327821 True True
407 red_1 Shoot at blue_0
418 red_1 Shoot at blue_0
447 blue_0 Splash :red_0
482 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.8989995920344721 86.10791164894657
blue_1 False False 1200 0.8930024587011388 -46.47866642745273
-------------------------- Scene: 0 --------------------------
163 blue_0 DOWN
370 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 370 -0.1010014254615441 -1.4053750164459375
blue_1 False False 370 -1.1049948921282107 -1.1772624125189737
----------------